# Differentially private federated GAN: an ablation study

The checkpoints are available via the [link](https://www.dropbox.com/sh/6b7pydwwokdtbmw/AAAHzI62wTfumSlaCfI0ikDta?dl=0).

## Setup 

### Imports

In [195]:
import os
from glob import glob
from operator import itemgetter
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.animation as mpa
import numpy as np
from celluloid import Camera
from IPython.display import HTML

### Environment

In [5]:
EXPERIMENT_DIR = "../experiments"

## Utilities

In [98]:
from itertools import groupby


def nearby_groups(arr, tol_digits=2):
    # split up sorted input array into groups if they're similar enough
    for (_, grp) in groupby(arr, lambda x: round(x, tol_digits)):
        # yield value from group that is closest to an integer
        yield sorted(grp, key=lambda x: abs(round(x) - x))[0]

## Visualization

In [245]:
def get_checkpoints():
    return glob("{}/*".format(EXPERIMENT_DIR))
print("Example experiment checkpoint: {}".format(get_checkpoints()[0]))

Example experiment checkpoint: ../experiments/1000_10_6_6_10_10_True_0.23944106906446763_0.8513794064392131


For the experiment checkpoint `1000_10_6_6_10_10_True_0.23944106906446763_0.8513794064392131`, the options are:

- the number of rounds: `1000`
- the number of clients per round: `10`
- the number of steps to train the client discriminator: `6`
- the number of steps to train the server generator: `6`
- the number of rounds after which the evaluation is run: `10`
- the number of rounds after which the sample images are saved: `10`
- whether to use differential privacy: `True`
- the $L_2$ norm clipping value for the delta added to the server discriminator after training on clients: `0.23944106906446763`
- the noise multiplier for added noise to the sum of the clipped discriminator weight deltas: `0.8513794064392131`


In [248]:
def get_characteristic_images(img_round, round_tolerance=20):
    checkpoints = get_checkpoints()
    characteristic_images = {
        "dp_l2_norm_clip": [],
        "dp_noise_multiplier": [],
        "image_path": [],
    }

    for checkpoint in checkpoints:
        dp_l2_norm_clip, dp_noise_multiplier = tuple(checkpoint.split("_")[-2:])

        dp_l2_norm_clip = float(dp_l2_norm_clip)
        dp_noise_multiplier = float(dp_noise_multiplier)

        candidates = []

        for image_path in glob("{}/**/*.png".format(checkpoint), recursive=True):
            image_round = int(image_path[:-4].split("_")[-1])

            if 0 <= img_round - image_round <= round_tolerance:
                candidates.append((image_round, image_path))
        if len(candidates) > 0:
            characteristic_images["dp_l2_norm_clip"].append(float(dp_l2_norm_clip))
            characteristic_images["dp_noise_multiplier"].append(
                float(dp_noise_multiplier)
            )
            characteristic_images["image_path"].append(
                max(candidates, key=itemgetter(0))[1]
            )

    return pd.DataFrame(characteristic_images)
    

In [249]:
get_characteristic_images(1000)

,dp_l2_norm_clip,dp_noise_multiplier,image_path
0,2.408976,0.386463,../experiments/1000_10_6_6_10_10_True_2.408975...
1,0.358939,0.128383,../experiments/1000_10_6_6_10_10_True_0.358938...
2,0.119396,0.064232,../experiments/1000_10_6_6_10_10_True_0.119395...
3,4.883389,0.012873,../experiments/1000_10_6_6_10_10_True_4.883389...
4,0.105667,0.001130,../experiments/1000_10_6_6_10_10_True_0.105666...
5,2.825813,0.784843,../experiments/1000_10_6_6_10_10_True_2.825812...
6,0.080358,0.005553,../experiments/1000_10_6_6_10_10_True_0.080357...


In [254]:
checkpoints_info = {
    "dp_l2_norm_clip": [],
    "dp_noise_multiplier": [],
}

for checkpoint in checkpoints:
    dp_l2_norm_clip, dp_noise_multiplier = tuple(checkpoint.split("_")[-2:])

    dp_l2_norm_clip = float(dp_l2_norm_clip)
    dp_noise_multiplier = float(dp_noise_multiplier)

    checkpoints_info["dp_l2_norm_clip"].append(dp_l2_norm_clip)
    checkpoints_info["dp_noise_multiplier"].append(dp_noise_multiplier)

sorted_dp_l2_norm_clip = list(
    nearby_groups(np.sort(checkpoints_info["dp_l2_norm_clip"]), tol_digits=1)
)
sorted_dp_noise_multiplier = list(
    nearby_groups(np.sort(checkpoints_info["dp_noise_multiplier"]), tol_digits=1)
)

fig = plt.figure()
ax = plt.gca()
camera = Camera(fig)
plt.xticks([-0.1, 5.0] + sorted_dp_l2_norm_clip)
plt.yticks([-0.1] + sorted_dp_noise_multiplier)

plt.grid(alpha=0.3)
plt.box(False)
plt.xticks(rotation=90)
plt.tick_params(axis="both", labelsize=10)
plt.ylabel("dp_noise_multiplier", fontsize=12, labelpad=15)
plt.xlabel("dp_l2_norm_clip", fontsize=12, labelpad=15)

xticks = ax.xaxis.get_major_ticks()
xticks[0].label1.set_visible(False)

yticks = ax.yaxis.get_major_ticks()
yticks[0].label1.set_visible(False)


def plot_characteristic_img(
    img_round, zoom=0.3, alpha=0.8, round_tolerance=20
):
    characteristic_images = get_characteristic_images(img_round)
    plt.text(
        max(sorted_dp_l2_norm_clip) / 2 - 1,
        1,
        "Characteristic images for rounds {}-{}".format(
            img_round - round_tolerance, img_round
        ),
        horizontalalignment="left",
        fontsize=14,
        fontweight="bold",
    )
    fig.set_size_inches(18.5, 10.5)
    fig.tight_layout()

    for idx, row in characteristic_images.iterrows():
        dp_l2_norm_clip = row["dp_l2_norm_clip"]
        dp_noise_multiplier = row["dp_noise_multiplier"]
        image_path = row["image_path"]
        im = OffsetImage(plt.imread(image_path), zoom=zoom, alpha=alpha)
        ab = AnnotationBbox(im, (dp_l2_norm_clip, dp_noise_multiplier), frameon=False)
        ax.add_artist(ab)


for img_round in np.linspace(100, 1000, 50, dtype=int):
    plot_characteristic_img(img_round)
    camera.snap()

animation = camera.animate()
animation.save("../docs/ablation_results.gif", writer="imagemagick", dpi=200)
plt.close()
HTML(animation.to_html5_video())